In [8]:
import os
import gc
import pandas as pd

PATH = '/home/kai/data/kaggle/talkingdata/haoyandata1/'
cols = ['os', 'hour', 'device', 'channel', 'app']
nrows = 10
train = pd.read_feather(PATH + 'train_cleaned.ftr')[cols].iloc[:nrows]
test = pd.read_feather(PATH + 'test_cleaned.ftr')[cols].iloc[:nrows]
for file in os.listdir(PATH):
    tmp = file.split('.')
    if (tmp[0] != 'train_cleaned' or tmp[0] != 'test_cleaned') and tmp[1] == 'ftr':
        ttmp = tmp[0].split('_')
        print(ttmp)
        if ttmp[0] == 'train': train['_'.join(ttmp[1:])] = pd.read_feather(PATH + file).iloc[:nrows]
        else: test['_'.join(ttmp[2:])] = pd.read_feather(PATH + file).iloc[:nrows]
        gc.collect()
        print(file)

['train', 'count', 'app', 'device', 'day', 'hour']
train_count_app_device_day_hour.ftr
['test', 'supplement', 'woe', 'app', 'os']
test_supplement_woe_app_os.ftr
['test', 'supplement', 'woe', 'device']
test_supplement_woe_device.ftr
['train', 'woe', 'ip', 'os', 'nextClickLeakDayFlt']
train_woe_ip_os_nextClickLeakDayFlt.ftr
['test', 'supplement', 'woe', 'app', 'channel']
test_supplement_woe_app_channel.ftr
['train', 'rang', 'count', 'ip', 'device', 'os', 'dayhourminute']
train_rang_count_ip_device_os_dayhourminute.ftr
['test', 'supplement', 'woe', 'channel']
test_supplement_woe_channel.ftr
['train', 'rang', 'count', 'ip', 'channel', 'dayhourminute']
train_rang_count_ip_channel_dayhourminute.ftr
['train', 'count', 'ip', 'app', 'device', 'os', 'day', 'hour']
train_count_ip_app_device_os_day_hour.ftr
['train', 'rang', 'count', 'ip', 'channel', 'dayhourminute10']
train_rang_count_ip_channel_dayhourminute10.ftr
['train', 'variance', 'ip', 'device', 'hour']
train_variance_ip_device_hour.ftr
['

KeyboardInterrupt: 

In [9]:
print(train)

   os  hour  device  channel  app  count_app_device_day_hour  \
0  13    22       1      379    3                         43   
1  19    22       1      379    3                         43   
2  13    22       1      379    3                         43   
3  13    22       1      478   14                          2   
4  13    22       1      379    3                         43   
5  16    22       1      379    3                         43   
6  23    22       1      379    3                         43   
7  19    22       1      379    3                         43   
8  13    22       1      379    3                         43   
9  22    22       1      459   64                          2   

   woe_ip_os_nextClickLeakDayFlt  rang_count_ip_device_os_dayhourminute  \
0                           -1.0                                   2328   
1                           -1.0                                   2374   
2                           -1.0                                    72

In [10]:
print(test)

   os  hour  device  channel  app    woe_app_os  woe_device  woe_app_channel  \
0  18    22       1      379    3 -7.374403e+02    0.535058     -1380.059706   
1  13    22       1      379    3 -1.273446e+02    0.535058     -1380.059706   
2  13    22       1      379    3 -1.273446e+02    0.535058     -1380.059706   
3  17    22       1      379    3 -6.778074e+02    0.535058     -1380.059706   
4  30    22       1      120   58          -inf    0.535058    -35740.839092   
5   8    22       1      379    3 -1.392030e+03    0.535058     -1380.059706   
6  13    22       1      379    3 -1.273446e+02    0.535058     -1380.059706   
7  13    22       1      379    3 -1.273446e+02    0.535058     -1380.059706   
8  34    22       1      120   58          -inf    0.535058    -35740.839092   
9  48    22       1      379    3 -6.637997e+04    0.535058     -1380.059706   

   woe_channel  rang_count_ip_day  rang_count_ip_hour  \
0  -318.726378                  3                  24   
1  -3